In [2]:
import gym
import numpy as np

env = gym.make('CartPole-v0')

goal_average_steps = 195
max_number_of_steps = 200
num_consecutive_iterations = 100
num_episodes = 5
last_time_steps = np.zeros(num_consecutive_iterations)


In [3]:
import tensorflow as tf

In [4]:
observation = env.reset()
#observation = np.expand_dims(observation,axis=0)
print(observation)


[-0.02899742  0.03581846 -0.01303823  0.00349526]


In [4]:
for i in range(10):
    #env.render()
    observations = env.step( 1 )
    print(observations)

(array([ 0.04161311,  0.38799624,  0.01878862, -0.55793345]), 1.0, False, {})
(array([ 0.04937303,  0.58284947,  0.00762995, -0.84463818]), 1.0, False, {})
(array([ 0.06103002,  0.77786648, -0.00926282, -1.134912  ]), 1.0, False, {})
(array([ 0.07658735,  0.97310841, -0.03196106, -1.43048555]), 1.0, False, {})
(array([ 0.09604952,  1.16861001, -0.06057077, -1.73298314]), 1.0, False, {})
(array([ 0.11942172,  1.36436866, -0.09523043, -2.04387955]), 1.0, False, {})
(array([ 0.14670909,  1.56033094, -0.13610802, -2.36444656]), 1.0, False, {})
(array([ 0.17791571,  1.75637668, -0.18339695, -2.69568782]), 1.0, False, {})
(array([ 0.21304325,  1.95230052, -0.23731071, -3.03826203]), 1.0, True, {})
(array([ 0.25208926,  2.14779176, -0.29807595, -3.39239689]), 0.0, True, {})


No handlers could be found for logger "gym.envs.classic_control.cartpole"


In [17]:
def policy_gradient():
    with tf.variable_scope("mypolicy",reuse=False):
        v = tf.get_variable("policy_parameters",[4,2])
        # state
        state = tf.placeholder("float",[None,4])
        # actions -1 or 1
        actions = tf.placeholder("float",[None,2])
        linear = tf.matmul(state,params)
        probabilities = tf.nn.softmax(linear)
        good_probabilities = tf.reduce_sum(tf.mul(probabilities, actions),reduction_indices=[1])
        # maximize the log probability
        log_probabilities = tf.log(good_probabilities)
        loss = -tf.reduce_sum(log_probabilities)
        optimizer = tf.train.AdamOptimizer(0.01).minimize(loss)

In [19]:
def value_gradient():  
    # sess.run(calculated) to calculate value of state
    with tf.variable_scope("mypolicy_grad",reuse=False):
        state = tf.placeholder("float",[None,4])
        w1 = tf.get_variable("w1",[4,10])
        b1 = tf.get_variable("b1",[10])
        h1 = tf.nn.relu(tf.matmul(state,w1) + b1)
        w2 = tf.get_variable("w2",[10,1])
        b2 = tf.get_variable("b2",[1])
        calculated = tf.matmul(h1,w2) + b2

        # sess.run(optimizer) to update the value of a state
        newvals = tf.placeholder("float",[None,1])
        diffs = calculated - newvals
        loss = tf.nn.l2_loss(diffs)
        optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

In [6]:
q_table = np.random.uniform(low=-1, high=1, size=(4 ** 4, env.action_space.n))

def bins(clip_min, clip_max, num):
    return np.linspace(clip_min, clip_max, num + 1)[1:-1]

def digitize_state(observation):
    # change status to 4 x 4 x 4 x 4 indices = [0 255]
    # then search matrix 256 x 2 actions..
    cart_pos, cart_v, pole_angle, pole_v = observation
    digitized = [np.digitize(cart_pos, bins=bins(-2.4, 2.4, 4)),
                 np.digitize(cart_v, bins=bins(-3.0, 3.0, 4)),
                 np.digitize(pole_angle, bins=bins(-0.5, 0.5, 4)),
                 np.digitize(pole_v, bins=bins(-2.0, 2.0, 4))]
    # [0  255]
    return sum([x * (4 ** i) for i, x in enumerate(digitized)])

In [7]:
digitize_state(observation)

86

In [8]:
q_table[150]

array([ 0.34599695,  0.76987585])

In [9]:
def get_action(state, action, observation, reward):
    next_state = digitize_state(observation)

    epsilon = 0.1
    r = np.random.uniform(0,1)
    if r < epsilon:
        next_action = np.argmax(q_table[next_state])
    else:
        next_action = np.random.choice([0,1])
        
    # update table ....
    alpha = 0.2
    gamma = 0.99
    q_table[state, action] = (1 - alpha) * q_table[state, action] +\
            alpha * (reward + gamma * q_table[next_state, next_action])

    return next_action, next_state

In [10]:
for episode in range(num_episodes):
    # init 
    observation = env.reset()

    state = digitize_state(observation)
    action = np.argmax(q_table[state])

    episode_reward = 0
    for t in range(max_number_of_steps):
        #env.render()

        # get observation , reward 
        observation, reward, done, info = env.step(action)

        if done:
            reward =- 200
        # behavior
        action, state = get_action(state, action, observation, reward)
        episode_reward += reward
        
        if done:
            print('%d Episode finished after %f time steps / mean %f' % (episode, t + 1,
                last_time_steps.mean()))
            last_time_steps = np.hstack((last_time_steps[1:], [episode_reward]))
            break

    if (last_time_steps.mean() >= goal_average_steps): # 直近の100エピソードが195以上であれば成功
        print('Episode %d train agent successfuly!' % episode)
        break
        
        

0 Episode finished after 62.000000 time steps / mean 0.000000
1 Episode finished after 34.000000 time steps / mean -1.390000
2 Episode finished after 33.000000 time steps / mean -3.060000
3 Episode finished after 18.000000 time steps / mean -4.740000
4 Episode finished after 13.000000 time steps / mean -6.570000
